In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d1 import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions

from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.read_wright_weights import save_weights
from utility.read_wright_weights import reload_weights
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

eval_env = MyEnv()
policies_own = {
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    "red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    "red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
policies_enem = {
    "red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    "red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
# policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config_own = ppo.DEFAULT_CONFIG.copy()
config_own = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 0,
          "train_batch_size": 1200*25,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_own,  "policy_mapping_fn": policy_mapping_fn}
         }
config_enem = ppo.DEFAULT_CONFIG.copy()
config_enem = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 600*5*10,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_enem,  "policy_mapping_fn": policy_mapping_fn}
         }

res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config_own,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

adversary = ppo.PPOTrainer(config=config_enem,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    
    #trainer.restore(checkpoint_path=model_path)
    #save_weights("blue_0",trainer)
    #save_weights("blue_1",trainer)

    reload_weights(policy_id="red_0",trainer=trainer,set_policy_id="blue_0")
    reload_weights(policy_id="red_1",trainer=trainer,set_policy_id="blue_1")
    reload_weights(policy_id="blue_0",trainer=trainer,set_policy_id="blue_0")
    reload_weights(policy_id="blue_1",trainer=trainer,set_policy_id="blue_1")
    save_weights("red_0",trainer)
    save_weights("red_1",trainer)


models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
hist_dir = os.path.join('./' + PROJECT + '/hist/')
if not os.path.exists(hist_dir):
    os.makedirs(hist_dir)

for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-04-03 14:49:40,156	INFO trainer.py:723 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2022-04-03 14:49:40,156	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by the mult

2022-04-03 14:49:41,793	WARNING deprecation.py:46 -- DeprecationWarning: `SampleBatch['is_training']` has been deprecated. Use `SampleBatch.is_training` instead. This will raise an error in the future!


2022-04-03 14:49:55,199	INFO trainable.py:127 -- Trainable.setup took 15.046 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-03 14:49:55,200	WARNING util.py:57 -- Install gputil for GPU system monitoring.


2022-04-03 14:50:08,566	INFO trainable.py:127 -- Trainable.setup took 13.319 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-03 14:50:08,568	WARNING util.py:57 -- Install gputil for GPU system monitoring.


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:

eval_env.reset()
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')

        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            trainer.restore(checkpoint_path=model_path)
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_1=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            action_dict1 = [0,0]
            rewards = {"blue_0":0,"blue_1":0}
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                action_dict0 = trainer.compute_single_action(obs['blue_0'],
                                                             state=state_0,prev_action=None,prev_reward=None,
                                                             policy_id='blue_0',explore=False)
                action_dict1 = trainer.compute_single_action(obs['blue_1'],
                                                             state=state_1,prev_action=None,prev_reward=None,
                                                             policy_id='blue_1',explore=False)

                state_0 = action_dict0[1]
                state_1 = action_dict1[1]
                obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0[0], 'blue_1': action_dict1[0]})

                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])


                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()

                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue",steps,env_blue_pos_mod,hist_dir)
                    save_hists("red",steps,env_red_pos_mod,hist_dir)
                    save_hists("mrm",steps,env_mrm_pos_mod,hist_dir)
                    save_weights("blue_0",trainer)
                    save_weights("blue_1",trainer)
                    # print(f'all done at {env.steps}')
                    break
                

            
            #if record_mode == 0:
               # video.release()

ray.shutdown()

-------------------------- Scene: 0 --------------------------

----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
226 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 1200 -10.101000968697019 -13.612807472916307
blue_1 False False 1200 -10.101003202030352 -113.98087790491763
-------------------------- Scene: 0 --------------------------
467 blue_1 Shoot at red_1 launch distance : 59627.26961836761 True True
485 blue_1 Shoot at red_1 launch distance : 53276.18223861915 True True
587 blue_1 Splash :red_1
TIME LIMIT LOSE
blue_0 False False 1200 -9.001004286864664 -20.2830512554426
blue_1 False False 1200 -9.101000820197997 -5.485224724777879
-------------------------- Scene: 0 --------------------------
TIME LIMIT LOSE
blue_0 False False 1200 -10.101001934356354 -62.898937163503106
blue_1 False False 1200 -10.101011834356354 -105.89633259683649
---------

TIME LIMIT LOSE
blue_0 False False 1200 -10.101001461836342 -67.87697768583159
blue_1 False False 1200 -10.100992028503008 -92.41146541916504
-------------------------- Scene: 0 --------------------------
TIME LIMIT LOSE
blue_0 False False 1200 -10.101001350726605 -49.63198135028185
blue_1 False False 1200 -10.101001350726605 -93.97697578361519
-------------------------- Scene: 0 --------------------------
318 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 1200 -10.101000087815951 -14.717826180833313
blue_1 False False 1200 -10.10100758781595 -82.62118753329591
-------------------------- Scene: 0 --------------------------
455 blue_0 Shoot at red_1 launch distance : 59530.32577081378 True True
478 blue_0 Shoot at red_1 launch distance : 54293.82382244201 True True
600 blue_0 Splash :red_1
TIME LIMIT LOSE
blue_0 False False 1200 -9.101000809720894 115.88821255388325
blue_1 False False 1200 -9.101000809720894 -98.83578337944923
-------------------------- Scene: 0 ------------------------

2022-04-03 15:41:20,548	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


agent_timesteps_total: 120764
custom_metrics: {}
date: 2022-04-03_16-12-25
done: false
episode_len_mean: 1161.1923076923076
episode_media: {}
episode_reward_max: 17.052429174435204
episode_reward_mean: -95.28150330957808
episode_reward_min: -168.79526976033958
episodes_this_iter: 26
episodes_total: 26
experiment_id: f890d766faf64bf888c22d72f9e74573
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 5.755937576293945
        entropy_coeff: 0.0
        kl: 0.088699571788311
        model: {}
        policy_loss: 0.024204012006521225
        total_loss: 50.93812942504883
        vf_explained_var: -0.12970423698425293
        vf_loss: 50.89618682861328
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 5.6881256103515625
        entropy_coeff: 0.0
   

2022-04-03 16:12:26,576	INFO trainable.py:468 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000001\checkpoint-1
2022-04-03 16:12:26,577	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 1, '_timesteps_total': 0, '_time_total': 4929.864478111267, '_episodes_total': 26}


...

2022-04-03 21:56:10,507	INFO trainable.py:468 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000005\checkpoint-5
2022-04-03 21:56:10,508	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 5, '_timesteps_total': 0, '_time_total': 25189.925378084183, '_episodes_total': 136}


-------------------------- Scene: 0 --------------------------
334 red_0 Shoot at blue_1
345 red_0 Shoot at blue_1
435 blue_1: Destroyed
570 red_1 Shoot at blue_0
581 red_1 Shoot at blue_0
683 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False True 683 -10.142247459789923 -12.844847387045757
blue_1 False True 683 -10.100998493123257 6.687858440733226

----------------- Training at steps:5 start! -----------------
-------------------------- Scene: 0 --------------------------
565 blue_0 DOWN
778 blue_1 Shoot at red_0 launch distance : 59750.393681441135 True True
800 blue_1 Shoot at red_0 launch distance : 51749.096892762915 True True
870 blue_1 Splash :red_0
TIME LIMIT LOSE
blue_0 False False 1200 -9.101000492341269 -11.956231296616197
blue_1 False False 1200 -9.101000492341269 78.4463967810348
-------------------------- Scene: 0 --------------------------
979 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 1200 -10.100999925095461 -15.15202754892695
blue_1 False False 1200 -10.100989625095

701 blue_1 Shoot at red_1 launch distance : 59770.715460691004 True True
739 red_1 Shoot at blue_1
750 red_1 Shoot at blue_1
764 blue_1 Splash :red_1
781 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 -9.101 -26.358752700000032
blue_1 False False 1200 -9.101 23.274609556544533
-------------------------- Scene: 0 --------------------------
190 blue_0 DOWN
599 red_1 Shoot at blue_1
610 red_1 Shoot at blue_1
672 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 672 -10.100999038981143 -11.404942077728958
blue_1 False False 672 -10.143547838981142 -10.852528572761837
-------------------------- Scene: 0 --------------------------
999 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 1200 -10.100999855421506 -60.525858935840766
blue_1 False False 1200 -10.001007988754838 -48.75395921307291
-------------------------- Scene: 0 --------------------------
805 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 1200 -10.101001154833845 -19.304947289791098
blue_1 False False 1200 -10.1010011

2022-04-03 23:21:04,995	INFO trainable.py:468 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000006\checkpoint-6
2022-04-03 23:21:04,996	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 6, '_timesteps_total': 0, '_time_total': 30218.57436323166, '_episodes_total': 163}


-------------------------- Scene: 0 --------------------------
319 red_1 Shoot at blue_0
325 red_0 Shoot at blue_0
330 red_1 Shoot at blue_0
336 red_0 Shoot at blue_0
396 blue_0: Destroyed
1018 blue_1 Shoot at red_0 launch distance : 75.27544964999299 True True
TIME LIMIT LOSE
blue_0 False False 1200 -9.601002037560496 47.695340070638785
blue_1 False False 1200 -9.407002037560495 59.40046458176112

----------------- Training at steps:6 start! -----------------
-------------------------- Scene: 0 --------------------------
1115 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 1200 -10.101000562260023 -49.496997977964924
blue_1 False False 1200 -10.101000562260023 -61.513004144029736
-------------------------- Scene: 0 --------------------------
225 blue_0 DOWN
510 blue_1 Shoot at red_1 launch distance : 59646.359276394454 True True
527 blue_1 Shoot at red_1 launch distance : 52144.810532134376 True True
591 blue_1 Splash :red_1
TIME LIMIT LOSE
blue_0 False False 1200 -9.101 -11.3714551637